# MuchLearningSuchWow - LSTM - Testing

This notebook contains the code we used to test our LSTM network. The training code is based primarily on [this kernel](https://www.kaggle.com/bountyhunters/baseline-lstm-with-keras-0-7).

### Imports & Data Paths

In [1]:
import numpy as np
import pandas as pd
import pickle
import time
import keras

import keras.backend as K

Using TensorFlow backend.


In [2]:
inputPath = "input/m5-forecasting-accuracy/"
outputPath = "output/"
modelPath = "models/"
submissionPath = "submissions/"

### Constants

In [3]:
timesteps = 14 # Number of previous days that will be used to predict the next day

### Loading Data

In [4]:
with open(outputPath + "/preprocessed_train_valid_data.pkl", "rb") as f:
    df_train_valid = pickle.load(f)
with open(outputPath + "/additional_features_testing.pkl", "rb") as f:
    additional_features_valid, additional_features_eval = pickle.load(f)
with open(outputPath + "/scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

In [5]:
df_sample_submission = pd.read_csv(inputPath + "/sample_submission.csv")

In [6]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

model = keras.models.load_model(modelPath + "/lstm_model_best", custom_objects = {'root_mean_squared_error' : root_mean_squared_error})

### Testing

In [7]:
def test_model(model, timesteps, input_data, scaler, nr_days_to_predict, additional_features):
    # Scale and convert input data so that it can be fed into the model
    inputs = scaler.transform(input_data)
    X_test = np.array([inputs])
    
    # Predict sales for the next nr_days_to_predict days
    predictions = []
    for j in range(0, nr_days_to_predict):
        predicted_sales = model.predict(X_test[:,-timesteps:,:].reshape(1, timesteps, 30490 + additional_features.shape[1]))
        test_input = np.column_stack((np.array(predicted_sales), additional_features.iloc[[j]]))
        test_input_expanded = np.expand_dims(test_input, 0)
        X_test = np.append(X_test, test_input_expanded, axis = 1)
        predicted_sales = scaler.inverse_transform(test_input)[:,0:30490]
        predictions.append(predicted_sales)
    
    return predictions

In [8]:
predictions_valid = test_model(model, timesteps, df_train_valid[-28-timesteps:-28], scaler, 28, additional_features_valid)
predictions_eval = test_model(model, timesteps, df_train_valid[-timesteps:], scaler, 28, additional_features_eval)

### Writing Submission File

In [9]:
submission_valid = pd.DataFrame(data=np.array(predictions_valid).reshape(28,30490))
submission_valid = submission_valid.T
submission_eval = pd.DataFrame(data=np.array(predictions_eval).reshape(28,30490))
submission_eval = submission_eval.T

In [10]:
submission = pd.concat((submission_valid, submission_eval), ignore_index=True)
idColumn = df_sample_submission[["id"]]
submission[["id"]] = idColumn  
cols = list(submission.columns)
cols = cols[-1:] + cols[:-1]
submission = submission[cols]
colsdeneme = ["id"] + [f"F{i}" for i in range (1,29)]
submission.columns = colsdeneme
submission.to_csv(submissionPath + "/submission.csv", index=False)